In [5]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# define the templates
car_template =""" You are an expert in automobiles. You have extensive knowledge
        about car mechanics, model, repair works and automative technology. You provide
        clear and helpfu; answers about cars.
        Here is the question:
        Question: {question}
       """
technology_template =""" You are a tech expert with in-depth knowledge on the latest gadgets, software and technology trends
You provide insightful and detailed answers about technology.
 Here is the question:
        Question: {question}
"""
restaurant_template = """You are a knowledgeable food critic and restaurant reviewer. You have a deep understanding of \
different cuisines, dining experiences, and what makes a great restaurant. You answer questions about restaurants insightfully.

Here is the question:
{question}"""

In [3]:
# Create some sample questions 
car_questions =[
    "What is the difference between a sedan and a SUV?"
    "How does a hybrid car save fuel?"
    "What should I look for when buying a used car?"
]

restaurant_questions =[
    "What makes a five-star restaurant exceptional?"
    "What's meant by happy hours?"
    "what are the key elements of French cuisine?"
]
technology_questions =[
    "What is the difference between a smartphone and a laptop computer?"
    "What is the difference between a server and a client?"
    "How do I secure my mobile from cyber threats?"
]


In [6]:
embedding = OpenAIEmbeddings()
car_question_embeddings = embedding.embed_documents(car_questions)
technology_question_embeddings = embedding.embed_documents(technology_questions)
restaurant_question_embeddings = embedding.embed_documents(restaurant_questions)


In [7]:

def prompt_router(input):
    query_embedding = embedding.embed_query(input["question"])
   
    car_similarity = cosine_similarity([query_embedding], car_question_embeddings)[0]
    restaurant_similarity = cosine_similarity([query_embedding], restaurant_question_embeddings)[0]
    technology_similarity = cosine_similarity([query_embedding], technology_question_embeddings)[0]
   
    max_similarity = max(max(car_similarity), max(restaurant_similarity), max(technology_similarity))
    if max_similarity == max(car_similarity):
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif max_similarity == max(restaurant_similarity):
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    elif max_similarity == max(technology_similarity):
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)

input_query ={"question":"what's the best way to improve my laptop's  battery life?"}
prompt = prompt_router(input_query)

Using TECHNOLOGY


In [14]:
chain =(
    {"question":RunnablePassthrough()}
    | RunnableLambda(prompt_router)   # Get the dynamic prompt here
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)   
    | StrOutputParser()
)

In [11]:
chain.invoke("How to identify the best calori food item?")

Using RESTAURANT


'Identifying the best calorie food item, particularly in the context of a restaurant menu, involves a few key considerations. Here’s a breakdown of how to approach this:\n\n1. **Understand Your Goals**: First, clarify what you mean by "best." Are you looking for the most calorie-dense item, the best balance of calories to nutrition, or perhaps the most satisfying dish for the calories it contains? Your goals will guide your choices.\n\n2. **Nutritional Value**: Look beyond just calories. A food item that is high in calories but low in nutrients (like empty calories from sugary desserts) may not be the best choice. Instead, consider items that provide a good balance of macronutrients (proteins, fats, carbohydrates) and micronutrients (vitamins and minerals). For example, a dish rich in healthy fats (like avocado or nuts) and protein (like grilled chicken or fish) can be calorie-dense yet nutritious.\n\n3. **Portion Size**: Evaluate the portion size in relation to the calorie count. A sm

In [12]:
chain.invoke("Langchain or LamaIndex?")

Using TECHNOLOGY


"When comparing Langchain and LamaIndex, it's essential to understand the context in which each tool is used, as they serve different purposes within the realm of AI and data management.\n\n### Langchain\n\n**Overview:**\nLangchain is a framework designed for building applications that utilize large language models (LLMs). It provides a structured way to integrate LLMs into various applications, enabling developers to create more complex and interactive AI-driven solutions.\n\n**Key Features:**\n1. **Modular Components:** Langchain offers a modular architecture, allowing developers to use different components like prompt templates, chains, and agents to build their applications.\n2. **Integration with LLMs:** It supports various LLMs, making it easier to switch between models or use multiple models in a single application.\n3. **Memory Management:** Langchain includes features for managing conversational memory, which is crucial for applications that require context retention over mult

In [13]:
chain.invoke("How to get good Mileage in driving?")

Using CAR


"Getting good mileage from your vehicle involves a combination of driving habits, vehicle maintenance, and understanding your car's mechanics. Here are some tips to help you maximize fuel efficiency:\n\n### Driving Habits\n\n1. **Smooth Acceleration and Braking**: Avoid rapid acceleration and hard braking. Gradual changes in speed can improve fuel efficiency.\n\n2. **Maintain a Steady Speed**: Use cruise control on highways to maintain a constant speed, which can help save fuel.\n\n3. **Limit Idling**: Turn off your engine if you expect to be stopped for more than a minute or two. Idling consumes fuel without moving the vehicle.\n\n4. **Drive at Moderate Speeds**: Fuel efficiency typically decreases at speeds above 50 mph. Try to keep your speed within the optimal range for your vehicle.\n\n5. **Plan Your Trips**: Combine errands into one trip to avoid multiple short trips, which can consume more fuel.\n\n### Vehicle Maintenance\n\n1. **Regular Maintenance**: Keep up with regular maint

## Using Classification

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
classification_template =PromptTemplate.from_template(
""" 
You are good at classifying a question.
Given the below question, classify it as either "Car" or "resturant" or "technology".

<If the question is about car mechanics, models, or automative technology,classify it as Car>
<If the question is about cuisines, dining experience, or resturant services, classify it as resturant>
<If the question is about technology, gadgets, or software, classify it as technology>

Question: {question}

Classification:
"""
)

In [21]:
classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [22]:
def prompt_router(input_query):
    # find the classification for the query
    classification =classification_chain.invoke({"question":input_query["question"]})
    if classification == "Car":
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif classification == "Resturant":
        print("Using Resturant")
        return PromptTemplate.from_template(restaurant_template)
    elif classification == "Technology":
        print("Using Technology")
        return PromptTemplate.from_template(technology_template)
    else:
        print("Using Default", classification)
        return None

input_query = {"question": "What is the capital of France?"}
prompt = prompt_router(input_query)

Using Default None of the above categories, it is a general knowledge question.


In [23]:
input_query ={"question":"Can increasing the RAM will increase the laptop performance?"}
prompt = prompt_router(input_query)


Using Technology


In [24]:
chain =(
    {"question":RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    |ChatOpenAI()
    |StrOutputParser()
)

In [25]:
chain.invoke(".Net or python?")

Using Technology


'Answer: The choice between .Net and Python really depends on the specific requirements of your project and your personal preferences.\n\n.Net is a framework developed by Microsoft that is widely used for building Windows applications, web applications, and services. It offers a wide range of tools and libraries that can help developers build robust and scalable applications. .Net is particularly popular in enterprise environments due to its integration with Microsoft products and services.\n\nOn the other hand, Python is a versatile and easy-to-learn programming language that is popular for its simplicity and readability. Python is often used for web development, data analysis, artificial intelligence, and scientific computing. It has a large and active community of developers who contribute to a vast ecosystem of libraries and frameworks.\n\nIf you are looking to build Windows applications or work in a Microsoft-centric environment, .Net may be the better choice for you. However, if 